In [ ]:
import nannyml as nml
from IPython.display import display

# Load synthetic data
reference_df, analysis_df, _ = nml.load_synthetic_car_loan_dataset()
display(reference_df.head())

,id,car_value,salary_range,debt_to_income_ratio,loan_length,repaid_loan_on_prev_car,size_of_downpayment,driver_tenure,repaid,timestamp,y_pred_proba,y_pred
0,0,39811.0,40K - 60K €,0.632950,19.0,False,40%,0.212653,1.0,2018-01-01 00:00:00.000,0.99,1
1,1,12679.0,40K - 60K €,0.718627,7.0,True,10%,4.927549,0.0,2018-01-01 00:08:43.152,0.07,0
2,2,19847.0,40K - 60K €,0.721724,17.0,False,0%,0.520817,1.0,2018-01-01 00:17:26.304,1.00,1
3,3,22652.0,20K - 20K €,0.705992,16.0,False,10%,0.453649,1.0,2018-01-01 00:26:09.456,0.98,1
4,4,21268.0,60K+ €,0.671888,21.0,True,30%,5.695263,1.0,2018-01-01 00:34:52.608,0.99,1


In [ ]:
print(reference_df.head().to_markdown(tablefmt="grid"))

+----+------+-------------+----------------+------------------------+---------------+---------------------------+-----------------------+-----------------+----------+-------------------------+----------------+----------+
|    |   id |   car_value | salary_range   |   debt_to_income_ratio |   loan_length | repaid_loan_on_prev_car   | size_of_downpayment   |   driver_tenure |   repaid | timestamp               |   y_pred_proba |   y_pred |
+====+======+=============+================+========================+===============+===========================+=======================+=================+==========+=========================+================+==========+
|  0 |    0 |       39811 | 40K - 60K €    |               0.63295  |            19 | False                     | 40%                   |        0.212653 |        1 | 2018-01-01 00:00:00.000 |           0.99 |        1 |
+----+------+-------------+----------------+------------------------+---------------+---------------------------+---

In [ ]:
non_feature_columns = ['timestamp', 'y_pred_proba', 'y_pred', 'repaid']

# Define feature columns
feature_column_names = [
    col for col in reference_df.columns
    if col not in non_feature_columns
]

calc = nml.DataReconstructionDriftCalculator(
    column_names=feature_column_names,
    timestamp_column_name='timestamp',
    chunk_size=5000
)
calc.fit(reference_df)
results = calc.calculate(analysis_df)

In [ ]:
display(results.filter(period='analysis').to_df())

chunk                                                        \
             key chunk_index start_index end_index          start_date   
0       [0:4999]           0           0      4999 2018-10-30 18:00:00   
1    [5000:9999]           1        5000      9999 2018-11-30 00:36:00   
2  [10000:14999]           2       10000     14999 2018-12-30 07:12:00   
3  [15000:19999]           3       15000     19999 2019-01-29 13:48:00   
4  [20000:24999]           4       20000     24999 2019-02-28 20:24:00   
5  [25000:29999]           5       25000     29999 2019-03-31 03:00:00   
6  [30000:34999]           6       30000     34999 2019-04-30 09:36:00   
7  [35000:39999]           7       35000     39999 2019-05-30 16:12:00   
8  [40000:44999]           8       40000     44999 2019-06-29 22:48:00   
9  [45000:49999]           9       45000     49999 2019-07-30 05:24:00   

                                    reconstruction_error            \
                 end_date    period       sampling_error     value   
0 2018-11-30 00:27:16.848  analysis             0.007972  1.550145   
1 2018-12-30 07:03:16.848  analysis             0.007972  1.582304   
2 2019-01-29 13:39:16.848  analysis             0.007972  1.620080   
3 2019-02-28 20:15:16.848  analysis             0.007972  1.662053   
4 2019-03-31 02:51:16.848  analysis             0.007972  1.703779   
5 2019-04-30 09:27:16.848  analysis             0.007972  1.616114   
6 2019-05-30 16:03:16.848  analysis             0.007972  1.630989   
7 2019-06-29 22:39:16.848  analysis             0.007972  1.672649   
8 2019-07-30 05:15:16.848  analysis             0.007972  1.692315   
9 2019-08-29 11:51:16.848  analysis             0.007972  1.742426   

                                                                       \
  upper_confidence_boundary lower_confidence_boundary upper_threshold   
0                  1.574061                  1.526230        1.538532   
1                  1.606220                  1.558388        1.538532   
2                  1.643996                  1.596164        1.538532   
3                  1.685969                  1.638138        1.538532   
4                  1.727695                  1.679863        1.538532   
5                  1.640030                  1.592198        1.538532   
6                  1.654905                  1.607074        1.538532   
7                  1.696565                  1.648733        1.538532   
8                  1.716231                  1.668400        1.538532   
9                  1.766341                  1.718510        1.538532   

                         
  lower_threshold alert  
0        1.431452  True  
1        1.431452  True  
2        1.431452  True  
3        1.431452  True  
4        1.431452  True  
5        1.431452  True  
6        1.431452  True  
7        1.431452  True  
8        1.431452  True  
9        1.431452  True

In [ ]:
from docs.utils import print_multi_index_markdown
print_multi_index_markdown(results.filter(period='analysis').to_df())

+----+---------------+-----------------+-----------------+---------------+---------------------+----------------------------+------------+--------------------------+-----------+-------------------------------+-------------------------------+---------------------+---------------------+-----------+
|    | | chunk       |                 |                 |               |                     |                            |            | | reconstruction_error   |           |                               |                               |                     |                     |           |
|    | | key         | | chunk_index   | | start_index   | | end_index   | | start_date        | | end_date                 | | period   | | sampling_error         | | value   | | upper_confidence_boundary   | | lower_confidence_boundary   | | upper_threshold   | | lower_threshold   | | alert   |
+====+===============+=================+=================+===============+=====================+==========

In [ ]:
display(results.filter(period='reference').to_df())

chunk                                                        \
             key chunk_index start_index end_index          start_date   
0       [0:4999]           0           0      4999 2018-01-01 00:00:00   
1    [5000:9999]           1        5000      9999 2018-01-31 06:36:00   
2  [10000:14999]           2       10000     14999 2018-03-02 13:12:00   
3  [15000:19999]           3       15000     19999 2018-04-01 19:48:00   
4  [20000:24999]           4       20000     24999 2018-05-02 02:24:00   
5  [25000:29999]           5       25000     29999 2018-06-01 09:00:00   
6  [30000:34999]           6       30000     34999 2018-07-01 15:36:00   
7  [35000:39999]           7       35000     39999 2018-07-31 22:12:00   
8  [40000:44999]           8       40000     44999 2018-08-31 04:48:00   
9  [45000:49999]           9       45000     49999 2018-09-30 11:24:00   

                                     reconstruction_error            \
                 end_date     period       sampling_error     value   
0 2018-01-31 06:27:16.848  reference             0.007972  1.502829   
1 2018-03-02 13:03:16.848  reference             0.007972  1.485016   
2 2018-04-01 19:39:16.848  reference             0.007972  1.475606   
3 2018-05-02 02:15:16.848  reference             0.007972  1.482116   
4 2018-06-01 08:51:16.848  reference             0.007972  1.454284   
5 2018-07-01 15:27:16.848  reference             0.007972  1.473131   
6 2018-07-31 22:03:16.848  reference             0.007972  1.468134   
7 2018-08-31 04:39:16.848  reference             0.007972  1.494718   
8 2018-09-30 11:15:16.848  reference             0.007972  1.494084   
9 2018-10-30 17:51:16.848  reference             0.007972  1.519998   

                                                                       \
  upper_confidence_boundary lower_confidence_boundary upper_threshold   
0                  1.526745                  1.478913        1.538532   
1                  1.508932                  1.461101        1.538532   
2                  1.499521                  1.451690        1.538532   
3                  1.506032                  1.458201        1.538532   
4                  1.478200                  1.430368        1.538532   
5                  1.497047                  1.449216        1.538532   
6                  1.492050                  1.444219        1.538532   
7                  1.518634                  1.470802        1.538532   
8                  1.518000                  1.470168        1.538532   
9                  1.543914                  1.496082        1.538532   

                          
  lower_threshold  alert  
0        1.431452  False  
1        1.431452  False  
2        1.431452  False  
3        1.431452  False  
4        1.431452  False  
5        1.431452  False  
6        1.431452  False  
7        1.431452  False  
8        1.431452  False  
9        1.431452  False

In [ ]:
print_multi_index_markdown(results.filter(period='reference').to_df())

+----+---------------+-----------------+-----------------+---------------+---------------------+----------------------------+------------+--------------------------+-----------+-------------------------------+-------------------------------+---------------------+---------------------+-----------+
|    | | chunk       |                 |                 |               |                     |                            |            | | reconstruction_error   |           |                               |                               |                     |                     |           |
|    | | key         | | chunk_index   | | start_index   | | end_index   | | start_date        | | end_date                 | | period   | | sampling_error         | | value   | | upper_confidence_boundary   | | lower_confidence_boundary   | | upper_threshold   | | lower_threshold   | | alert   |
+====+===============+=================+=================+===============+=====================+==========

In [ ]:
figure = results.plot()
figure.show()

In [ ]:
figure.write_image('../_static/tutorials/detecting_data_drift/multivariate_drift_detection/pca-reconstruction-error.svg')

In [ ]:
non_feature_columns = ['timestamp', 'y_pred_proba', 'y_pred', 'repaid']

feature_column_names = [
    col for col in reference_df.columns
    if col not in non_feature_columns
]

from sklearn.impute import SimpleImputer

calc = nml.DataReconstructionDriftCalculator(
    column_names=feature_column_names,
    timestamp_column_name='timestamp',
    chunk_size=5000,
    imputer_categorical=SimpleImputer(strategy='constant', fill_value='missing'),
    imputer_continuous=SimpleImputer(strategy='median')
)
calc.fit(reference_df)
results = calc.calculate(analysis_df)
